In [1]:
import sys
sys.path.append('/home/ph/LLM2/swift')
sys.path.insert(0,'/home/ph/LLM2/swift')

测试qwen-vl-chat

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

from swift.llm import (
    get_model_tokenizer, get_template, inference, ModelType, get_default_template_type,
)
from swift.utils import seed_everything

model_type = ModelType.qwen_vl_chat
template_type = get_default_template_type(model_type)
print(f'template_type: {template_type}')  # template_type: qwen

model, tokenizer = get_model_tokenizer(model_type, model_kwargs={'device_map': 'auto'})

template = get_template(template_type, tokenizer)
seed_everything(42)
query = tokenizer.from_list_format([
    {'image': 'https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg'},
    {'text': '这是什么'},
])
response, history = inference(model, template, query)
print(f'query: {query}')
print(f'response: {response}')
query = '输出击掌的检测框'
response, history = inference(model, template, query, history)
print(f'query: {query}')
print(f'response: {response}')
print(f'history: {history}')
image = tokenizer.draw_bbox_on_latest_picture(response, history)
image.save('output_chat.jpg')

/home/ph/anaconda3/envs/torch2.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-06 13:05:32,572 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-05-06 13:05:32,574 - modelscope - INFO - Loading ast index from /home/ph/.cache/modelscope/ast_indexer
2024-05-06 13:05:32,611 - modelscope - INFO - Loading done! Current index file version is 1.10.0, with md5 f6e226a8578971e8fd7923bb0e86893b and a total number of 946 components indexed
[INFO:swift] Successfully registered `/home/ph/LLM2/swift/swift/llm/data/dataset_info.json`
2024-05-06 13:05:33,254	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
[INFO:swift] Loading the model using model_dir: /home/ph/LLM2/VL/Qwen-VL-Chat
[INFO:swift] Setting torch

template_type: qwen


Loading checkpoint shards: 100%|██████████| 10/10 [00:05<00:00,  1.89it/s]
[INFO:swift] model.max_model_len: 2048
[INFO:swift] Global seed set to 42


query: Picture 1: <img>https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg</img>
这是什么
response: 图中是一名女子在沙滩上和狗玩耍，旁边的狗是一只拉布拉多犬，它们处于沙滩上。
query: 输出击掌的检测框
response: <ref>击掌</ref><box>(523,513),(584,605)</box>
history: [['Picture 1: <img>https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg</img>\n这是什么', '图中是一名女子在沙滩上和狗玩耍，旁边的狗是一只拉布拉多犬，它们处于沙滩上。'], ['输出击掌的检测框', '<ref>击掌</ref><box>(523,513),(584,605)</box>']]


# sft

测试sft更改认知

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '6'
import sys
sys.path.append('/home/ph/LLM2/swift')
sys.path.insert(0,'/home/ph/LLM2/swift')
from swift.llm import DatasetName, ModelType, SftArguments, sft_main

sft_args = SftArguments(
    model_type=ModelType.qwen_14b_chat,
    # model_id_or_path='/home/ph/LLM/Qwen1.5/Qwen1.5-14B-Chat-GPTQ-Int4',  model_cache_dir='/home/ph/LLM/Qwen1.5/Qwen1.5-14B-Chat-GPTQ-Int4',
    dataset=[DatasetName.sharegpt_gpt4_mini],
    train_dataset_sample=1000,
    logging_steps=5,
    max_length=2048,
    warmup_ratio=0.4,
    output_dir='output',
    lora_target_modules=['ALL'],   # 在所有的linear层(包括qkvo以及mlp)加lora. 这通常是效果最好的.
    self_cognition_sample=500,
    model_name=['猴乐乐', 'houlele'],
    model_author=['同元智算', 'Tongyuan intelligence calculation'])
output = sft_main(sft_args)
best_model_checkpoint = output['best_model_checkpoint']
print(f'best_model_checkpoint: {best_model_checkpoint}')

## datasets加载

In [1]:
# 自定义
import os
from modelscope.msdatasets import MsDataset
custom_cache_dir='/home/ph/LLM2/swift/datasets'
if not os.path.exists(custom_cache_dir):
    os.makedirs(custom_cache_dir)
ds = MsDataset.load(dataset_name='AI-ModelScope/alpaca-gpt4-data-zh',subset_name='default' ,split="train",
                    download_mode= 'reuse_dataset_if_exists',cache_dir=custom_cache_dir)
print(next(iter(ds)))

2024-05-06 16:43:16,469 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-05-06 16:43:16,471 - modelscope - INFO - Loading ast index from /home/ph/.cache/modelscope/ast_indexer
2024-05-06 16:43:16,518 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 b97acdcdf6b803b3e9e523c04105cb52 and a total number of 972 components indexed
/home/ph/anaconda3/envs/torch2.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-05-06 16:43:18,497 - modelscope - WARNING - Authentication has expired, please re-login if you need to access private models or datasets.
2024-05-06 16:43:18,881 - modelscope - INFO - dataset_type: 4
2024-05-06 16:43:18,885 - modelscope - WARNING - Authentication has expired, please re-login if you need to access private models or datasets.
2024-05-06 16:43:1

{'instruction': '保持健康的三个提示。', 'input': None, 'output': '以下是保持健康的三个提示：\n\n1. 保持身体活动。每天做适当的身体运动，如散步、跑步或游泳，能促进心血管健康，增强肌肉力量，并有助于减少体重。\n\n2. 均衡饮食。每天食用新鲜的蔬菜、水果、全谷物和脂肪含量低的蛋白质食物，避免高糖、高脂肪和加工食品，以保持健康的饮食习惯。\n\n3. 睡眠充足。睡眠对人体健康至关重要，成年人每天应保证 7-8 小时的睡眠。良好的睡眠有助于减轻压力，促进身体恢复，并提高注意力和记忆力。'}


In [1]:
import os
from modelscope.msdatasets import MsDataset
custom_cache_dir='/home/ph/LLM2/swift/datasets'
if not os.path.exists(custom_cache_dir):
    os.makedirs(custom_cache_dir)

ds = MsDataset.load("alpaca-gpt4-data-zh", namespace="AI-ModelScope", split="train",cache_dir=custom_cache_dir)
print(next(iter(ds)))

2024-05-06 16:41:53,357 - modelscope - INFO - PyTorch version 2.2.1 Found.
2024-05-06 16:41:53,360 - modelscope - INFO - Loading ast index from /home/ph/.cache/modelscope/ast_indexer
2024-05-06 16:41:53,410 - modelscope - INFO - No valid ast index found from /home/ph/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-05-06 16:41:53,480 - modelscope - INFO - Loading done! Current index file version is 1.13.3, with md5 b97acdcdf6b803b3e9e523c04105cb52 and a total number of 972 components indexed
/home/ph/anaconda3/envs/torch2.1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ImportError: cannot import name 'RepoFolder' from 'huggingface_hub.hf_api' (/home/ph/anaconda3/envs/torch2.1/lib/python3.9/site-packages/huggingface_hub/hf_api.py)